In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import AUC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [76]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [77]:
test_ids = test_data['ID_code']

In [78]:
train_data = train_data.drop(columns=['ID_code'])
test_data = test_data.drop(columns=['ID_code'])
train_data.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [20]:
X = train_data.drop(columns=['target'])
Y = train_data['target']

In [23]:
standardScaler = StandardScaler()
standardScaler.fit(X)
stdscaled_X = standardScaler.transform(X)
stdscaled_test = standardScaler.transform(test_data)

minmaxScaler = MinMaxScaler()
minmaxScaler.fit(X)
mmscaled_X = minmaxScaler.transform(X)
mmscaled_test = minmaxScaler.transform(test_data)

In [70]:
std_x_train, std_x_test, std_y_train, std_y_test = train_test_split(stdscaled_X, Y, test_size=0.2)
mm_x_train, mm_x_test, mm_y_train, mm_y_test = train_test_split(mmscaled_X, Y, test_size=0.2)

In [72]:
def create_model(dense_layers=1):
    model = Sequential()
    model.add(Dense(200,activation='relu'))
    for _ in range(dense_layers):
        model.add(Dense(250,activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['AUC'])
    return model

early_stopping = EarlyStopping(monitor='val_AUC', mode='max', patience=20, restore_best_weights=True)

In [66]:
model1 = create_model(dense_layers=1)
model1.fit(std_x_train, std_y_train, validation_data=(std_x_test, std_y_test), epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 4s 731us/step - AUC: 0.8096 - loss: 0.2611 - val_AUC: 0.8432 - val_loss: 0.2511
Epoch 2/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 3s 692us/step - AUC: 0.8553 - loss: 0.2345 - val_AUC: 0.8464 - val_loss: 0.2444
Epoch 3/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 4s 720us/step - AUC: 0.8680 - loss: 0.2255 - val_AUC: 0.8437 - val_loss: 0.2499
Epoch 4/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 4s 733us/step - AUC: 0.8855 - loss: 0.2099 - val_AUC: 0.8386 - val_loss: 0.2534
Epoch 5/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 4s 699us/step - AUC: 0.9034 - loss: 0.1937 - val_AUC: 0.8271 - val_loss: 0.2678
Epoch 6/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 3s 686us/step - AUC: 0.9203 - loss: 0.1753 - val_AUC: 0.8212 - val_loss: 0.2838
Epoch 7/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 4s 707us/step - AUC: 0.9343 - loss: 0.1608 - val_AUC: 0.8079 - val_loss: 0.2985
Epoch 8/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 3s 696us/step - AUC: 0.9477 - loss: 0.1447 - val_AUC: 0.7979 - val_loss: 0.3252
Epoch 9/50
5000/5000 ━━━

In [67]:
model2 = create_model(dense_layers=2)
model2.fit(std_x_train, std_y_train, validation_data=(std_x_test, std_y_test), epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - AUC: 0.7997 - loss: 0.2651 - val_AUC: 0.8467 - val_loss: 0.2451
Epoch 2/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - AUC: 0.8580 - loss: 0.2333 - val_AUC: 0.8450 - val_loss: 0.2486
Epoch 3/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - AUC: 0.8667 - loss: 0.2278 - val_AUC: 0.8454 - val_loss: 0.2461
Epoch 4/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - AUC: 0.8803 - loss: 0.2175 - val_AUC: 0.8381 - val_loss: 0.2579
Epoch 5/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - AUC: 0.8961 - loss: 0.2008 - val_AUC: 0.8309 - val_loss: 0.2644
Epoch 6/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - AUC: 0.9113 - loss: 0.1831 - val_AUC: 0.8238 - val_loss: 0.2716
Epoch 7/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - AUC: 0.9264 - loss: 0.1679 - val_AUC: 0.8156 - val_loss: 0.3115
Epoch 8/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - AUC: 0.9367 - loss: 0.1556 - val_AUC: 0.8073 - val_loss: 0.2899
Epoch 9/50
5000/5000 ━━━━━━━━━━━━━━━━━━━

In [68]:
model3 = create_model(dense_layers=3)
model3.fit(std_x_train, std_y_train, validation_data=(std_x_test, std_y_test), epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - AUC: 0.8018 - loss: 0.2642 - val_AUC: 0.8438 - val_loss: 0.2462
Epoch 2/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.8564 - loss: 0.2341 - val_AUC: 0.8456 - val_loss: 0.2472
Epoch 3/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.8619 - loss: 0.2314 - val_AUC: 0.8431 - val_loss: 0.2587
Epoch 4/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.8757 - loss: 0.2188 - val_AUC: 0.8396 - val_loss: 0.2527
Epoch 5/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.8904 - loss: 0.2083 - val_AUC: 0.8352 - val_loss: 0.2567
Epoch 6/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.9048 - loss: 0.1921 - val_AUC: 0.8275 - val_loss: 0.2616
Epoch 7/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.9192 - loss: 0.1786 - val_AUC: 0.8148 - val_loss: 0.2697
Epoch 8/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.9302 - loss: 0.1669 - val_AUC: 0.8022 - val_loss: 0.2961
Epoch 9/50
5000/5000 ━━━━━━━━━━━━━━━━━━━

In [69]:
model4 = create_model(dense_layers=4)
model4.fit(std_x_train, std_y_train, validation_data=(std_x_test, std_y_test), epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - AUC: 0.7971 - loss: 0.2666 - val_AUC: 0.8488 - val_loss: 0.2437
Epoch 2/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.8521 - loss: 0.2367 - val_AUC: 0.8469 - val_loss: 0.2487
Epoch 3/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.8641 - loss: 0.2275 - val_AUC: 0.8455 - val_loss: 0.2441
Epoch 4/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.8755 - loss: 0.2205 - val_AUC: 0.8437 - val_loss: 0.2480
Epoch 5/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.8840 - loss: 0.2113 - val_AUC: 0.8390 - val_loss: 0.2500
Epoch 6/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.8973 - loss: 0.1981 - val_AUC: 0.8287 - val_loss: 0.2693
Epoch 7/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.9076 - loss: 0.1880 - val_AUC: 0.8321 - val_loss: 0.2578
Epoch 8/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.9198 - loss: 0.1751 - val_AUC: 0.8198 - val_loss: 0.2887
Epoch 9/50
5000/5000 ━━━━━━━━━━━━━━

In [74]:
std_model = model4
mm_model = create_model(dense_layers=4)
mm_model.fit(mm_x_train, mm_y_train, validation_data=(mm_x_test, mm_y_test), epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - AUC: 0.7342 - loss: 0.2874 - val_AUC: 0.8570 - val_loss: 0.2440
Epoch 2/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.8366 - loss: 0.2454 - val_AUC: 0.8588 - val_loss: 0.2355
Epoch 3/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.8419 - loss: 0.2418 - val_AUC: 0.8589 - val_loss: 0.2366
Epoch 4/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.8409 - loss: 0.2441 - val_AUC: 0.8588 - val_loss: 0.2460
Epoch 5/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.8432 - loss: 0.2417 - val_AUC: 0.8593 - val_loss: 0.2374
Epoch 6/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - AUC: 0.8457 - loss: 0.2404 - val_AUC: 0.8594 - val_loss: 0.2367
Epoch 7/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.8441 - loss: 0.2411 - val_AUC: 0.8592 - val_loss: 0.2358
Epoch 8/50
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.8466 - loss: 0.2421 - val_AUC: 0.8589 - val_loss: 0.2362
Epoch 9/50
5000/5000 ━━━━━━━━━━━━━━

In [115]:
pred = mm_model.predict(mmscaled_test)
y_pred = [(1 if p>=0.5 else 0) for p in pred]
submission = pd.DataFrame({'ID_Code':test_ids, 'target':y_pred})
submission.to_csv('submission.csv', index=False)

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 3s 463us/step
